In [2]:
import pandas as pd
import numpy as np
import wrds
#import os
import time
#from tqdm import notebook as tq
#import copy

db = wrds.Connection()

Enter your WRDS username [DRNPRO]:nikoldy
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
db.list_libraries()
#db.list_tables(library='crsp_a_ccm')
db.describe_table(library="comp", table="fundq")
#db.list_tables(library='crspm')
#db.describe_table(library="crsp", table="dsf")
#db.describe_table(library="crsp_a_ccm", table="fundq")[0:20]

Approximately 1841123 rows in comp.fundq.


,name,nullable,type
0,gvkey,True,VARCHAR(24)
1,datadate,True,DATE
2,fyearq,True,DOUBLE PRECISION
3,fqtr,True,DOUBLE PRECISION
4,fyr,True,DOUBLE PRECISION
...,...,...,...
642,mkvaltq,True,DOUBLE PRECISION
643,prccq,True,DOUBLE PRECISION
644,prchq,True,DOUBLE PRECISION
645,prclq,True,DOUBLE PRECISION


In [2]:
start = time.time()
link_table = db.raw_sql("""select *
                        from crsp_a_ccm.ccmxpf_lnkhist 
                        """, date_cols=['linkenddt','linkdt'])
print('Elapsed time (seconds):',(time.time() - start))
link_table.head()

Elapsed time (seconds): 5.729357719421387


,gvkey,linkprim,liid,linktype,lpermno,lpermco,linkdt,linkenddt
0,001000,C,00X,NU,NaN,NaN,1961-01-01,1970-09-29
1,001000,P,01,NU,NaN,NaN,1970-09-30,1970-11-12
2,001000,P,01,LU,25881.0,23369.0,1970-11-13,1978-06-30
3,001001,C,00X,NU,NaN,NaN,1978-01-01,1983-09-19
4,001001,P,01,LU,10015.0,6398.0,1983-09-20,1986-07-31


In [ ]:
# get crsp data
start = time.time()
crsp_table = db.raw_sql("""select permno, hsiccd, date, prc, ret, shrout, cfacpr, cfacshr, retx
                        from crsp.dsf
                        where date>='01/01/1960'
                        and hsiccd not between 6000 and 7000
                        """, date_cols=['date'])

crsp_table.to_csv('data/crsp.csv')
print('Elapsed time (seconds):',((time.time() - start)/60))

In [149]:
# get compustat data
comp_table = db.raw_sql("""select gvkey, iid, datadate, datacqtr, fyearq, fqtr, fyr, tic, conm, 
                        atq, ltq, niq, ceqq, cheq, dlrsn, sic
                        from comp.fundq a
                        where datadate >='01/01/1960'
                        and sic not between 6000 and 7000
                        """, date_cols=['datadate','datacqtr'])
crsp_table.to_csv('data/crsp.csv')
print('Elapsed time (seconds):',((time.time() - start)/60))

,gvkey,iid,datadate,datacqtr,fyearq,fqtr,fyr,tic,conm,atq,ltq,niq,ceqq,cheq
0,001410,01,2019-01-31,2018-10-01,2019.0,1.0,10.0,ABM,ABMINDUSTRIESINC,3686.400,2225.300,13.000,1461.100,30.600
1,001562,01,2019-01-31,2018-10-01,2018.0,3.0,4.0,AMSWA,AMERICANSOFTWARE-CLA,160.821,46.465,2.301,114.356,83.164
2,001618,01,2019-01-31,2018-10-01,2018.0,3.0,4.0,AXR,AMREPCORP,104.510,16.539,-0.032,87.971,14.233
3,001632,01,2019-01-31,2018-10-01,2019.0,1.0,10.0,ADI,ANALOGDEVICES,21828.278,10242.841,355.006,11585.437,605.864
4,001704,01,2019-01-31,2018-10-01,2019.0,1.0,10.0,AMAT,APPLIEDMATERIALSINC,18922.000,10713.000,771.000,8209.000,3712.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79940,260846,01,2020-12-31,2020-10-01,2020.0,4.0,12.0,HTD,HANCOCKJOHNTAX-ADVDIVINC,NaN,NaN,NaN,NaN,NaN
79941,260847,01,2020-12-31,2020-10-01,2020.0,4.0,12.0,IGR,CBRECLARIONGLOBALREINCM,NaN,NaN,NaN,NaN,NaN
79942,260853,01,2020-12-31,2020-10-01,2020.0,4.0,12.0,WIW,WESTERNASSINFL-LINKEDOPPT,NaN,NaN,NaN,NaN,NaN
79943,264686,01,2020-12-31,2020-10-01,2020.0,4.0,12.0,UTF,COHEN&STEERSINFRASTRFD,NaN,NaN,NaN,NaN,NaN


In [147]:
failedmerge = db.raw_sql("""select a.gvkey, a.iid, a.datadate, c.date, a.datacqtr, a.fyearq, a.fqtr,
                        a.fyr, a.tic, a.conm,a.atq, a.ltq, a.niq, a.ceqq, a.cheq, c.permno, c.hsiccd, 
                        c.prc, c.ret, c.cfacpr, c.ret, c.retx
                        from comp.fundq a
                        join crsp_a_ccm.ccmxpf_lnkhist b on
                        (a.gvkey = b.gvkey and a.datadate = b.linkenddt and a.iid = b.liid)
                        join crsp.dsf c on
                        (b.lpermno = c.permno and b.linkdt = c.date)
                        """, date_cols=['a.datadate','c.date','datacqtr'])
failedmerge

,gvkey,iid,datadate,date,datacqtr,fyearq,fqtr,fyr,tic,conm,...,niq,ceqq,cheq,permno,hsiccd,prc,ret,cfacpr,ret,retx
0,001015,01,1986-10-31,1982-01-29,1986-07-01,1987.0,1.0,7.0,4327B,ADIELECTRONICSINC,...,-0.802,1.425,0.391,66405.0,3679.0,-1.6875,0.038462,1.000000,0.038462,0.038462
1,001044,01,1970-06-30,1968-01-31,1970-04-01,1970.0,2.0,12.0,AMK.3,AMKCORP,...,NaN,NaN,NaN,37591.0,2649.0,86.7500,-0.082011,2.000000,-0.082011,-0.082011
2,001076,01,1992-12-31,1982-11-04,1992-10-01,1992.0,3.0,3.0,PRG,PROGHOLDINGSINC,...,1.628,50.600,0.085,10517.0,7359.0,-20.6250,NaN,19.058825,NaN,NaN
3,001097,02,1994-09-30,1987-09-30,1994-07-01,1994.0,3.0,12.0,ACMTA,ACMATCORP-CLA,...,1.306,39.972,16.667,11647.0,6350.0,16.0000,0.103448,1.000000,0.103448,0.103448
4,001156,01,1975-06-30,1968-10-17,1975-04-01,1975.0,2.0,12.0,AOX.1,AERODEXINC,...,-0.203,0.610,NaN,46973.0,3729.0,26.2500,NaN,1.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,206468,90,2009-12-31,1998-02-17,2009-10-01,2009.0,4.0,12.0,ROSYY,ROSTELECOMPJSC,...,7.153,2027.585,725.216,85793.0,4812.0,20.6875,NaN,1.000000,NaN,NaN
514,239356,90,2014-06-30,2012-03-01,2014-04-01,2014.0,2.0,12.0,PRISY,PROMOTORADEINFORMACIONES,...,-2961.322,-702.237,255.295,12382.0,2711.0,3.6400,-0.016216,1.000000,-0.016216,-0.016216
515,260775,90,2013-12-31,2004-03-04,2013-10-01,2013.0,4.0,12.0,LTONY,MNCMEDIAINVESTMENTLTD,...,-9.922,139.973,114.411,90087.0,7374.0,17.4200,NaN,1.000000,NaN,NaN
516,295786,01,2013-09-30,2013-09-30,2013-07-01,2013.0,3.0,12.0,CNHI,CNHINDUSTRIALNV,...,223.000,NaN,NaN,14144.0,3531.0,12.5000,NaN,1.000000,NaN,NaN


In [133]:
spy = db.raw_sql("""
                SELECT date, permno, hexcd, hsiccd, prc, ret, cfacpr,openprc, retx, adjprc
                FROM crsp.dsf
                WHERE permno = 84398
                """,date_cols=['date'])
spy.head() # spy only has 

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "adjprc" does not exist
LINE 2: ...ermno, hexcd, hsiccd, prc, ret, cfacpr,openprc, retx, adjprc
                                                                 ^
HINT:  Perhaps you meant to reference the column "dsf.prc".

[SQL: 
                SELECT date, permno, hexcd, hsiccd, prc, ret, cfacpr,openprc, retx, adjprc
                FROM crsp.dsf
                WHERE permno = 84398
                ]
(Background on this error at: http://sqlalche.me/e/f405)

In [55]:
apple = db.raw_sql("""select permno, date, prc, ret, shrout 
                        from crsp.dsf 
                        where permno = 14593
                        and date>='01/01/2019'""", 
                     date_cols=['date'])

apple 

,permno,date,prc,ret,shrout
0,14593.0,2019-01-02,157.919998,0.001141,4729803.0
1,14593.0,2019-01-03,142.190002,-0.099607,4729803.0
2,14593.0,2019-01-04,148.259995,0.042689,4729803.0
3,14593.0,2019-01-07,147.929993,-0.002226,4729803.0
4,14593.0,2019-01-08,150.750000,0.019063,4729803.0
...,...,...,...,...,...
247,14593.0,2019-12-24,284.269989,0.000951,4443265.0
248,14593.0,2019-12-26,289.910004,0.019840,4443265.0
249,14593.0,2019-12-27,289.799988,-0.000379,4443265.0
250,14593.0,2019-12-30,291.519989,0.005935,4443265.0


In [80]:
apple_fund = db.raw_sql("""select a.gvkey, a.iid, a.datadate, a.tic, a.conm,
                            a.at, b.prccm, b.cshoq 
                            
                            from comp.funda a 
                            inner join comp.secm b 
                            
                            on a.gvkey = b.gvkey
                            and a.iid = b.iid
                            and a.datadate = b.datadate
                        
                            where a.tic = 'AAPL' 
                            and a.datadate>='01/01/2010'
                            and a.datafmt = 'STD' 
                            and a.consol = 'C' 
                            and a.indfmt = 'INDL'
                            """, date_cols=['datadate'])

apple_fund

,gvkey,iid,datadate,tic,conm,at,prccm,cshoq
0,001690,01,2010-09-30,AAPL,APPLEINC,75183.0,283.750,915.970
1,001690,01,2011-09-30,AAPL,APPLEINC,116371.0,381.320,929.277
2,001690,01,2012-09-30,AAPL,APPLEINC,176064.0,667.105,939.208
3,001690,01,2013-09-30,AAPL,APPLEINC,207000.0,476.750,899.213
4,001690,01,2014-09-30,AAPL,APPLEINC,231839.0,100.750,5866.161
5,001690,01,2015-09-30,AAPL,APPLEINC,290479.0,110.300,5578.753
6,001690,01,2016-09-30,AAPL,APPLEINC,321686.0,113.050,5336.166
7,001690,01,2017-09-30,AAPL,APPLEINC,375319.0,154.120,5126.201
8,001690,01,2018-09-30,AAPL,APPLEINC,365725.0,225.740,4754.986
9,001690,01,2019-09-30,AAPL,APPLEINC,338516.0,223.970,4443.236
